<a href="https://www.kaggle.com/code/shedai/llm-rag-2-kurumsal-dokuman-yonetimi?scriptVersionId=249533404" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Kurumsal Bilgi Yönetimi

Amaç: Bir çalışanın, şirketin iç yönetmelikleri (izin, masraf vb.) hakkında sorduğu bir soruya, ilgili dokümanlardan cevap bulmak.

In [2]:
# Gerekli kütüphaneler: pip install sentence-transformers scikit-learn numpy
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# --- BİLGİ KAYNAĞI ---
KNOWLEDGE_BASE = """
Madde 12: Yıllık İzin Politikası. Şirketimizde bir tam yılı dolduran her çalışan 14 iş günü ücretli yıllık izin hakkına sahiptir. Beş yıl ve üzeri kıdeme sahip çalışanlar için bu süre 20 iş günüdür. Kullanılmayan izinler bir sonraki yıla devretmez, ancak şirket yönetimi onayı ile en fazla 5 günü devredebilir.

Madde 13: Masraf Beyanı Politikası. Çalışanlar, şirket için yaptıkları seyahat, konaklama ve yemek masraflarını ayın son Cuma gününe kadar insan kaynakları departmanına bildirmelidir. Faturaların asıllarının beyana eklenmesi zorunludur. Onaylanan masraflar bir sonraki ayın maaşı ile birlikte ödenir.

Madde 14: Uzaktan Çalışma Politikası. Haftada iki gün uzaktan çalışma hakkı, departman yöneticisinin onayı ile tüm çalışanlara tanınmıştır. Uzaktan çalışılan günlerde çalışanların standart mesai saatleri içinde ulaşılabilir olması beklenir.
"""

# --- MODÜLER FONKSİYONLAR ---

def chunk_text(text: str) -> list[str]:
    """Metni paragraflarına göre böler."""
    return [p.strip() for p in text.split('\n\n') if p.strip()]

def get_embeddings(chunks, model):
    return model.encode(chunks)

def find_most_similar_chunks(query, embeddings, chunks, model, top_k=1):
    query_embedding = model.encode([query])
    similarities = cosine_similarity(query_embedding, embeddings)[0]
    top_k_indices = np.argsort(similarities)[-top_k:][::-1]
    return [chunks[i] for i in top_k_indices]

def generate_answer(query, context):
    prompt = f"Bağlam: {context}\n\nSoru: {query}\n\nCevap:"
    print("--- LLM'e Gönderilen Nihai Prompt ---\n", prompt)
    mock_response = f"Şirket politikasına göre, '{query}' sorunuzun cevabı şudur: {context}"
    return mock_response

# --- ANA PROGRAM AKIŞI ---
if __name__ == "__main__":
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
    user_query = "5 seneden uzun süredir çalışıyorum, yıllık iznim ne kadar?"
    
    print("--- Kurumsal Bilgi Yönetimi RAG Demosu ---")
    print(f"Kullanıcı Sorusu: '{user_query}'\n")

    # 1. Parçalama
    chunks = chunk_text(KNOWLEDGE_BASE)
    print(f"--- 1. Parçalama Sonucu: {len(chunks)} adet chunk (madde) bulundu. ---\n")

    # 2. Gömme
    document_embeddings = get_embeddings(chunks, embedding_model)

    # 3. Arama
    context_chunks = find_most_similar_chunks(user_query, document_embeddings, chunks, embedding_model)
    context_for_llm = "\n".join(context_chunks)
    print("--- 2. Arama Sonucu: En Alakalı Madde Bulundu ---\n", context_for_llm, "\n")

    # 4. Üretim
    final_answer = generate_answer(user_query, context_for_llm)
    print("\n--- 3. Nihai Cevap Üretildi ---")
    print(final_answer)

2025-07-09 05:24:11.548607: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752038651.794763      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752038651.868149      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

--- Kurumsal Bilgi Yönetimi RAG Demosu ---
Kullanıcı Sorusu: '5 seneden uzun süredir çalışıyorum, yıllık iznim ne kadar?'

--- 1. Parçalama Sonucu: 3 adet chunk (madde) bulundu. ---



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

--- 2. Arama Sonucu: En Alakalı Madde Bulundu ---
 Madde 12: Yıllık İzin Politikası. Şirketimizde bir tam yılı dolduran her çalışan 14 iş günü ücretli yıllık izin hakkına sahiptir. Beş yıl ve üzeri kıdeme sahip çalışanlar için bu süre 20 iş günüdür. Kullanılmayan izinler bir sonraki yıla devretmez, ancak şirket yönetimi onayı ile en fazla 5 günü devredebilir. 

--- LLM'e Gönderilen Nihai Prompt ---
 Bağlam: Madde 12: Yıllık İzin Politikası. Şirketimizde bir tam yılı dolduran her çalışan 14 iş günü ücretli yıllık izin hakkına sahiptir. Beş yıl ve üzeri kıdeme sahip çalışanlar için bu süre 20 iş günüdür. Kullanılmayan izinler bir sonraki yıla devretmez, ancak şirket yönetimi onayı ile en fazla 5 günü devredebilir.

Soru: 5 seneden uzun süredir çalışıyorum, yıllık iznim ne kadar?

Cevap:

--- 3. Nihai Cevap Üretildi ---
Şirket politikasına göre, '5 seneden uzun süredir çalışıyorum, yıllık iznim ne kadar?' sorunuzun cevabı şudur: Madde 12: Yıllık İzin Politikası. Şirketimizde bir tam yılı 